In [0]:
import libs.rename_columns as rc

In [0]:
path_source = '/Volumes/landing/raw/rfb_cnpj'
catalog = 'workspace'
schema  = 'bronze'
tables  = ['cnaes', 'empresas', 'estabelecimentos', 'socios', 'motivos', 'municipios', 'naturezas', 'paises', 'qualificacoes', 'simples']

In [0]:
for table in tables:
    
    table_columns = rc.columns(table)

    # realizando a leitura dos arquivos e criando um df spark
    df = (spark.read
        .option('delimiter', ';')
        .option('header', False)
        .option('inferSchema', False)
        .option('encoding', 'latin1')
        .csv(f"{path_source}/{table}*")
    )

    # Renomear as colunas
    for old, new in table_columns.items():
        df = df.withColumnRenamed(old, new)

    # Escrevendo em formato delta
    (df.write
        .mode('overwrite')
        .format('delta')
        .saveAsTable(f"{catalog}.{schema}.{table}")
    )

    print(table)
